In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras import utils
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
import keras

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut
from sklearn.utils import shuffle 

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

from pathlib import Path
import os.path
import sys
import datetime
import time



In [ ]:
import telegram
from keras.callbacks import Callback
from callbacks import TelegramCallback
from callbacks.TelegramData import TelegramData


# create callback
config = {
    'token': TelegramData.Token,   # paste your bot token
    'telegram_id': TelegramData.ID,                                   # paste your telegram_id
    'model_name': 'JAN_LSTM_CNN'
}

tg_callback = TelegramCallback(config)

In [ ]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

        tensor =  tf.convert_to_tensor(self.settings_str)
        summary = tf.summary.text ("Run_Settings", tensor)

        with  tf.Session() as sess:
            s = sess.run(summary)
            self.writer.add_summary(s)

In [ ]:
train_ids = [1,2,3, 7, 8, 9, 10,12,13,14,15,16,17]
test_ids = [4,5,6,11]

In [ ]:
dfAll = pd.read_pickle("PklData/df_lstm_norm30.pkl")
dfCNN =  pd.read_pickle("PklData/df_blobs.pkl")

In [ ]:
dfAll.TaskID[dfAll.TaskID>=17] = dfAll.TaskID - 17


In [ ]:
#dfNew = dfAll[dfAll.TaskID.isin([0, 2, 4,6,8,9,10,11,13,15])].copy()
#y = pd.Series([0, 2, 4,6,8,9,10,11,13,15], index=[0,1,2,3,4,5,6,7,8,9])
#dfNew.TaskID = dfNew.TaskID.replace([0, 2, 4,6,8,9,10,11,13,15], [0,1,2,3,4,5,6,7,8,9])
#dfAll = dfNew

In [ ]:
df_train = dfAll[dfAll.userID.isin(train_ids)]
df_test = dfAll[dfAll.userID.isin(test_ids)]

df_train2 = df_train[['Blobs', 'TaskID']].copy()
df_test2 = df_test[['Blobs', 'TaskID']].copy()

In [ ]:
df_train3 = dfCNN[dfCNN.userID.isin(train_ids)]
df_test3 = dfCNN[dfCNN.userID.isin(test_ids) & (dfCNN.Version == "Normal")]


df_train4 = df_train3[['Blobs', 'InputMethod']].copy()
df_test4 = df_test3[['Blobs', 'InputMethod']].copy()

In [ ]:
x_train3 = np.vstack(df_train4.Blobs)
x_test3 = np.vstack(df_test4.Blobs)
y_train3 = df_train4.InputMethod.values
y_test3 = df_test4.InputMethod.values

x_train3 = x_train3.reshape(-1, 27, 15, 1)
x_test3 = x_test3.reshape(-1, 27, 15, 1)

In [ ]:
x_train = np.concatenate(df_train2.Blobs.values).reshape(-1,30,27,15,1)
x_test = np.concatenate(df_test2.Blobs.values).reshape(-1,30,27,15,1)

y_train = df_train2.TaskID.values
y_test = df_test2.TaskID.values



In [ ]:
# convert class vectors to binary class matrices (one-hot notation)
num_classes = len(dfAll.TaskID.unique()) 
y_train_one_hot = utils.to_categorical(df_train2.TaskID, num_classes)
y_test_one_hot = utils.to_categorical(df_test2.TaskID, num_classes)

In [ ]:
# convert class vectors to binary class matrices (one-hot notation)
num_classes_cnn = 2
y_train_one_hot_cnn = utils.to_categorical(df_train3.InputMethod, num_classes_cnn)
y_test_one_hot_cnn = utils.to_categorical(df_test3.InputMethod, num_classes_cnn)

In [ ]:
import pydot

In [ ]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
#config = tf.ConfigProto(device_count = {"GPU": 1})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.3

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [ ]:
sess = K.get_session()
[node.op.name for node in KnuckelFinger.outputs]

In [ ]:
KnuckelFinger.input

In [ ]:
B = Model(inputs=KnuckelFinger.input, outputs=KnuckelFinger.output)

In [ ]:
x_train3.shape

In [ ]:
KnuckelFinger = load_model("10_01_19.h5", compile=True)

In [ ]:
type(KnuckelFinger)

In [ ]:

a = KnuckelFinger.predict_classes(x_train3)

In [ ]:
asdf = x_train3[144].reshape((1,27,15,1))
KnuckelFinger.predict_classes(asdf)

In [ ]:
print([i for i in range(len(a)) if a[i] ==1])

In [ ]:

KnuckelFinger.summary()

hist = KnuckelFinger.fit(x_test3, y_test_one_hot_cnn)

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

data = np.zeros((1,784))
labels = np.zeros((1,10))
# This returns a tensor
inputs = Input(shape=(784,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(64, activation='relu')(inputs)
x = Dense(64, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels)  # starts training

x = Input(shape=(784,))
# This works, and returns the 10-way softmax we defined above.
y = model(x)


In [ ]:
model.predict

In [ ]:


from keras.layers import TimeDistributed

# Input tensor for sequences of 20 timesteps,
# each containing a 784-dimensional vector
input_sequences = Input(shape=(30,27,15,1))

# This applies our previous model to every timestep in the input sequences.
# the output of the previous model was a 10-way softmax,
# so the output of the layer below will be a sequence of 20 vectors of size 10.
model = Model(inputs = KnuckelFinger.layers[0].input, outputs = KnuckelFinger.layers[-1].output)
processed_sequences = TimeDistributed(model)(input_sequences)
model2 = Model(inputs=input_sequences, outputs=processed_sequences)
type(model2)
model2.predict

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import plot_model
from tensorflow.keras.models import save_model, load_model
import pydot

batch_size = 60
epochs = 100
timesteps = 30
data_dim = (27,15)




tf.get_default_graph()
model = Sequential()

#### Gesture structure
A0 = Input(shape=(30,27,15,1),name='A0')
A1 = TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', 
                            kernel_regularizer=regularizers.l2(0.01)),input_shape=(30, 27,15,1), name='A1')(A0)
A2 = TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),name='A2')(A1)
#model.add(TimeDistributed(BatchNormalization(axis=-1)))
A3 = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last'),name='A3')(A2)
A4 = TimeDistributed(Dropout(0.20), name='A4')(A3)


A5 = TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),name='A5')(A4)
A6 = TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01)),name='A6')(A5)
#model.add(TimeDistributed(BatchNormalization(axis=-1)))
A7 = TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last'),name='A7')(A6)
A8 = TimeDistributed(Dropout(0.10), name='A8')(A7)

A9 = TimeDistributed(Flatten(), name='A9')(A8)

A10 = LSTM(128, return_sequences=True, input_shape=(timesteps, data_dim), name='A10', kernel_regularizer=regularizers.l2(0.01))(A9)
A11 = LSTM(64, kernel_regularizer=regularizers.l2(0.01), name='A11')(A10)
A12 = Dense(num_classes, activation='softmax', name='A12')(A11)
A = Model(inputs=A0, outputs=A12)

### END Gesture structure

#print(KnuckelFinger.outputs[0])

B0 = TimeDistributed(KnuckelFinger(), name='B0')(A0)
#B1 = Reshape((27,15,-1),name='B1')(A0)

#B2 = Conv2D(64 ,kernel_size=(3,3), input_shape=(30,27,15,1),activation='relu',name='B2')(B1)
#B3 = Flatten(name='B3')(B2)
#B3 = Dense(128, activation='relu', kernel_regularizer=keras.regularizers.L1L2(0.02, 0.15),name='B3', use_bias=True)(B2)
B4 = Dense(2, activation='softmax',name='B4')(B3)

merged = Model(inputs=[A0],outputs=[A12, B3])
#merged.add_weight(KnuckelFinger, shape=(27,15))
merged.summary()
optimizer = optimizers.Adam(lr = 0.0001, decay=1e-6)
#optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
merged.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
history = merged.fit(x_train, [y_train_one_hot, y_train_one_hot_cnn],
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test_one_hot))
                        #callbacks=[storer,logger,tg_callback, learning_rate_reduction])

#plot_model(merged,to_file='demo.png',show_shapes=True)

In [ ]:
from keras.layers import Input, Dense
batch_size = 60
epochs = 100
timesteps = 30
data_dim = (27,15)

tf.get_default_graph()
model = Sequential()
model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu', 
            padding='same', kernel_regularizer=regularizers.l2(0.01)), input_shape=(30, 27,15,1)))
model.add(TimeDistributed(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))))
#model.add(TimeDistributed(BatchNormalization(axis=-1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.20)))

model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))))
model.add(TimeDistributed(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(0.01))))
#model.add(TimeDistributed(BatchNormalization(axis=-1)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2,2), strides=None, padding='same', data_format='channels_last')))
model.add(TimeDistributed(Dropout(0.10)))

model.add(TimeDistributed(Flatten()))

model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, data_dim), kernel_regularizer=regularizers.l2(0.01)))
model.add(LSTM(64, kernel_regularizer=regularizers.l2(0.01)))

#### Finger Knuckle


model.add(Dense(num_classes, activation='softmax'))

#optimizer = optimizers.Adagrad()
optimizer = optimizers.Adam(lr = 0.0001, decay=1e-6)
#optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.1)
model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

# Broadcast progress to the tensorboard.
readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_LSTM_Jan_" + readable_timestamp

config = ""
for layer in model.layers:
    config += str(layer.output).split('\"')[1].split("/")[0] + str(layer.output_shape) + "\n\n"
config += "batchsize: " + str(batch_size) + "\n\n" + "epochs: " + str(epochs) + "\n\n"

model.summary()

readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')
tensorflowfolder = "/srv/share/tensorboardfiles/KnuckleFinger_LSTM_Jan_" + readable_timestamp
print("KnuckleFinger_LSTM_Jan_" + readable_timestamp)
logger = LoggingTensorBoard(settings_str_to_log = config, log_dir=tensorflowfolder, histogram_freq=0,
                            write_graph=True, write_images=True, update_freq = 'epoch')
storer = ModelCheckpoint("./ModelSnapshots/KnuckleFinger_LSTM_Jan_" + readable_timestamp + ".h5", monitor='val_loss', verbose=0,
                         save_best_only=True, save_weights_only=False, mode='auto', period=1)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=20, 
                                            verbose=1, 
                                            factor=0.95, 
                                            min_lr=0.00001)

history = model.fit(x_train, y_train_one_hot,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_test, y_test_one_hot),
                        callbacks=[storer,logger,tg_callback, learning_rate_reduction])


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['val_acc'], label="Test Accuracy")
plt.plot(history.history['acc'], label="Training Accuracy")
plt.legend()


In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    model.save('lstm_23_01_19.h5')